In [14]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches


def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    found_name = False
    for i in range(len(possible_names[0])):
        if possible_names[0][i] == name:
            new_mv, valid_zero = find_in_transfermarkt(name, season, opposing_country_code, transfermarkt_data)
            new_row = game_row.to_dict()  # Convert row to dict
            new_row['Market Value'] = new_mv
            new_row['Lookup Still Required?'] = "FALSE"
            new_row['Lookup Return Case'] = valid_zero
            new_rows.append(new_row)
            found_name = True
            break
    if not found_name:
        new_row = game_row.to_dict()
        new_row['Wrong Name'] = "Yes"
        new_row['Found Name'] = possible_names[0]
        wrong_rows.append(new_row)
    new_df = pd.DataFrame(new_rows)
    wrong_df = pd.DataFrame(wrong_rows)
    return new_df, wrong_df


leagues_value_old = load_csv_dataset('MarketValues.csv')
leagues_value_old = leagues_value_old[leagues_value_old['Season'] <= 2013]

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53856/1888214061.py:49: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [20]:
large_combined_df = load_csv_dataset('combined_DF_merged_April_23_New.csv')
large_combined_df = large_combined_df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53856/1888214061.py:49: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [21]:
#filter large combined DF 
large_combined_df_working = large_combined_df[(large_combined_df['Lookup Return Case'] == 'working') | 
                                              (large_combined_df['Lookup Return Case'] == 'Found in Curr Season') | 
                                              (large_combined_df['Lookup Return Case'] == 'Used Large Dataset')]

large_combined_df_not_working = large_combined_df[(large_combined_df['Lookup Return Case'] != 'working') &
                                              (large_combined_df['Lookup Return Case'] != 'Found in Curr Season') &
                                              (large_combined_df['Lookup Return Case'] != 'Used Large Dataset')].reset_index()

large_combined_df_not_working['Lookup Return Case'].value_counts()

Lookup Return Case
Only Before                                         4188
Used Average of Season Before and After             2712
Had Both but Both Were Empty                        2221
Used Season After * .8 But Had Both                 1057
Only After                                          1032
Only Before + Zero Before                            373
Both Empty                                           336
Used Season Before * 1.2                             163
Only After + Zero After                              153
Did Not Find in Curr Season, One Up, or One Down     115
Used Season Before * 1.2 But Had Both                 84
Used Season After * 1.2 But Had Both                  80
Used Season Before * .8                               31
Used Season Before * .8 But Had Both                  14
Name: count, dtype: int64

In [ ]:
#check other team 

    #need the row 

In [24]:
for i in range(0, 1): #len(large_combined_df_not_working)
    row = large_combined_df_not_working.loc[i]
    0==0
    #MAKE SURE ITS A BAD EDGE CASES

    #thing 1 to try

    #thing 2

    #thing 3 